# Practical Statistics for Data Scientists

Applied exercises from the O'Reilly book by Peter Bruce, Andrew Bruce, and Peter Gedeck. Each section demonstrates a core statistical concept with real data — focusing on intuition and practical interpretation over mathematical derivation.

**Topics covered:** Estimates of location, estimates of variability, percentiles and distributional shape.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import trim_mean
from wquantiles import median as w_median

sns.set_theme()

## Estimates of Location

Location estimates answer: "What is a typical value?" The choice of metric depends on the distribution shape and what "typical" means in context.

- **Mean**: arithmetic average — uses every observation, sensitive to outliers.
- **Trimmed Mean**: mean after dropping a fixed percentage of extremes — a compromise between mean and median.
- **Median**: the 50th percentile — resistant to outliers, ignores magnitudes.
- **Weighted Mean/Median**: each observation contributes proportionally to its weight (e.g., population size).

In [ ]:
# Dataset from the book's official GitHub repository
STATE_URL = "https://raw.githubusercontent.com/gedeck/practical-statistics-for-data-scientists/master/data/state.csv"

state = pd.read_csv(STATE_URL)
print(f"Dataset: {state.shape[0]} US states, {state.shape[1]} variables")
print(f"Columns: {', '.join(state.columns)}\n")
state.head()

,State,Population,Murder.Rate,Abbreviation
0,Alabama,4779736,5.7,AL
1,Alaska,710231,5.6,AK
2,Arizona,6392017,4.7,AZ


In [ ]:
population = state['Population']

mean_val = population.mean()
trimmed_mean_val = trim_mean(population, proportiontocut=0.1)
median_val = population.median()

print(f"Mean:          {mean_val:>12,.0f}")
print(f"Trimmed Mean:  {trimmed_mean_val:>12,.0f}  (10% trim)")
print(f"Median:        {median_val:>12,.0f}")
print(f"\nMean / Median ratio: {mean_val / median_val:.2f}")

Mean: 6162876.30, Trimmed Mean: 4783697.12, and Median: 4436369.50


The mean is pulled upward by a few very populous states (California, Texas, New York), landing well above the median. The trimmed mean sits between the two — robust to extremes without fully discarding them. A mean/median ratio significantly above 1.0 is a quick diagnostic for right-skewness.

In [ ]:
murder_rate = state['Murder.Rate']

weighted_mean = np.average(murder_rate, weights=population)
weighted_median = w_median(murder_rate, weights=population)
unweighted_mean = murder_rate.mean()
unweighted_median = murder_rate.median()

print(f"Murder Rate — Unweighted Mean:   {unweighted_mean:.2f}")
print(f"Murder Rate — Weighted Mean:     {weighted_mean:.2f}  (weighted by population)")
print(f"Murder Rate — Unweighted Median: {unweighted_median:.2f}")
print(f"Murder Rate — Weighted Median:   {weighted_median:.2f}  (weighted by population)")

Weighted Mean: 4.45, Weighted Median: 4.40


Weighting by population shifts both metrics upward, meaning more populous states tend to have higher murder rates. The weighted mean rises more than the weighted median — again because the mean is more sensitive to large values introduced by high-population, high-murder-rate states.

## Estimates of Variability

Variability estimates answer: "How spread out are the values?" Different metrics capture different aspects of spread and have different robustness properties.

- **Standard Deviation**: average distance from the mean — most common, but sensitive to outliers.
- **Interquartile Range (IQR)**: width of the middle 50% (Q3 - Q1) — resistant to extreme values.
- **Median Absolute Deviation (MAD)**: median of absolute deviations from the median — the most robust measure.

In [ ]:
std_val = population.std()
iqr_val = population.quantile(0.75) - population.quantile(0.25)
mad_val = (population - population.median()).abs().median()

print(f"Standard Deviation:        {std_val:>12,.0f}")
print(f"Interquartile Range (IQR): {iqr_val:>12,.0f}")
print(f"Median Absolute Deviation: {mad_val:>12,.0f}")
print(f"\nStd / MAD ratio: {std_val / mad_val:.2f}")

The standard deviation is over 2x the MAD, reflecting the outsized influence of a few very large states on the mean-based metric. The IQR tells us the middle half of states span roughly a few million in population — a more representative picture of typical state-to-state variation than the standard deviation alone.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot
axes[0].boxplot(population, vert=True, widths=0.5)
axes[0].set_ylabel('Population')
axes[0].set_title('State Population Distribution')

# Annotate outliers
q1, q3 = population.quantile(0.25), population.quantile(0.75)
upper_fence = q3 + 1.5 * (q3 - q1)
outliers = state[population > upper_fence][['State', 'Population']].sort_values('Population', ascending=False)
for _, row in outliers.iterrows():
    axes[0].annotate(row['State'], xy=(1, row['Population']), fontsize=8,
                     xytext=(1.15, row['Population']),
                     arrowprops=dict(arrowstyle='-', color='gray'))

# Histogram with location markers
axes[1].hist(population, bins=20, color='steelblue', edgecolor='black', linewidth=0.5)
axes[1].axvline(population.median(), color='red', linestyle='--', label=f'Median = {population.median():,.0f}')
axes[1].axvline(population.mean(), color='orange', linestyle='--', label=f'Mean = {population.mean():,.0f}')
axes[1].set_xlabel('Population')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Population Histogram with Location Estimates')
axes[1].legend()

plt.tight_layout()
plt.show()

## Percentiles and Distributional Shape

Percentiles divide the ordered data into equal parts and are the foundation of non-parametric summary statistics. The full percentile table reveals the distribution shape more richly than any single metric.

In [ ]:
percentiles = [5, 10, 25, 50, 75, 90, 95]
pct_values = np.percentile(population, percentiles)

pct_df = pd.DataFrame({
    'Percentile': [f'{p}th' for p in percentiles],
    'Population': [f'{v:,.0f}' for v in pct_values]
})
print(pct_df.to_string(index=False))

ratio_95_5 = pct_values[-1] / pct_values[0]
print(f"\n95th / 5th percentile ratio: {ratio_95_5:.1f}x")
print("This confirms heavy right-skewness: the largest states are far more populous than the smallest.")

## Takeaways

- **No single metric tells the full story.** The mean and standard deviation describe the data well when the distribution is roughly symmetric, but mislead for skewed data like state populations. Always pair them with median/IQR or visual inspection.
- **Weighted estimates shift the perspective.** Unweighted estimates treat each state equally; weighted estimates reflect the experience of the average *person*. The right choice depends on the question being asked.
- **Percentile tables are underused.** A 7-number summary (5th through 95th) provides a fast, complete picture of any univariate distribution and should be standard in any EDA.